In [1]:
import os

from IDRnD.utils import seed_everything
from IDRnD.augmentations import ToMellSpec, PadOrClip, ToTensor, Normalize_predef
from IDRnD.dataset import Test_Dataset
from IDRnD.resnet import resnet34
from IDRnD.pipeline import *


import pandas as pd
import numpy as np
import librosa
import torch
from torch.utils.data import DataLoader
from torchvision.transforms import transforms

seed_everything(0)

In [2]:
post_transform = transforms.Compose([
    ToMellSpec(n_mels=128),
    Normalize_predef(-29.6179, 16.6342),
    librosa.power_to_db,
    PadOrClip(320),
    ToTensor(),
])

In [4]:
X, y = get_train_data()

NameError: name 'get_train_data' is not defined

In [5]:
test_dataset = Test_Dataset(X)

NameError: name 'X' is not defined

In [8]:
hm = Train()

#test_dataset = Test_Dataset(np.array(eval_protocol["path"]), post_transform)

#test_loader = DataLoader(test_dataset, batch_size=100, shuffle=False)

model = resnet34(num_classes=1).cuda()

model.eval()
model.load_state_dict(torch.load("models/resnet_34_better_val.pt"))
pred = hm.predict_on_test(3, model)

TypeError: 'int' object is not iterable